In [ ]:
!pip install langchain #langchain
!pip install pincone-client #to connect to the pincone client
!pip install pypdf #To load the pdf data

In [ ]:
!pip install openai
!pip install tiktoken

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [ ]:
import os

In [ ]:
#Load the pdf
!mkdir pdfs

!gdown
!gdown

In [ ]:
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()

In [ ]:
data

In [ ]:
#Chunking time

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500,chunk_overlap = 30)
text_chunks = text_splitter.split_documents(data)

In [ ]:
len(text_chunks)

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = "PUT your KEY"

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
res = embeddings.embed_query("Hi")

In [ ]:
len(res) #1536

In [ ]:
#Pinecone API and Environment

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY','')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV','')

In [ ]:
#Initialise pinecone

import pinecone
pinecone.init(
    api_key = PINECONE_API_KEY,
    environment=PINECONE_API_ENV
)

index_name = "VectorDB" #Create an index on pinecone website,can name it any.

In [ ]:
search = Pinecone.from_texts([t.page_content for t in text_chunks],embeddings,index_name=index_name) #list comprehension in python

In [ ]:
#Loading pre-existing index/cluster

search = Pinecone.from_existing_index(index_name,embeddings)
search

In [ ]:
query = "What is GDP?"

In [ ]:
docs = search.similarity_search(query,k=3)
docs

In [ ]:
#LLM wrapper
llm = OpenAI()

q_a = RetrievalQA.from_chain_type(llm = llm,chain_type="stuff",retriever=search.as_retriever())

In [ ]:
q_a.run(query)

In [ ]:
import sys
while True:
  user_input = input(f"Input Prompt")
  if user_input == 'exit':
    print("Exiting")
    sys.exit()
  if user_input =='':
    continue
  result =q_a({'query': user_input})
  print(f"Answer : {result['result']}")